In [ ]:
import sys, os, pathlib
sys.path.append("/root/shared/gitrepos/smart-comp-sci/utils")
import smart_analysis
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
params = {'axes.labelsize': 12,
            'axes.titlesize': 6,
            'legend.fontsize': 10,
            'xtick.labelsize':10,
            'ytick.labelsize': 10,
            'figure.figsize': (6,4),
            'pdf.fonttype': 42,
            'ps.fonttype': 42,
            'axes.spines.top': False,
            'axes.spines.right': False,
            'legend.loc': "right"}
matplotlib.rcParams.update(params)

In [ ]:
cur_dir = "/root/shared/gitrepos/smart-comp-sci/cru-example"
cru_results_folder = "/root/scratch/smart-comp-sci-data/cru_results_massconserv/cru_fixedSERCA"
cru_results_folder_noSERCA = "/root/scratch/smart-comp-sci-data/cru_results_massconserv/cru_noSERCA"
cru_mesh = "/root/scratch/smart-comp-sci-data/cru_results_massconserv/cru_mesh/cru_mesh.h5"
results_folders = [cru_results_folder, cru_results_folder, cru_results_folder,
                   cru_results_folder_noSERCA, cru_results_folder_noSERCA, cru_results_folder_noSERCA]
tests = ["cru-all", "cru-center", "cru-edge", 
         "cru-all-noSERCA", "cru-center-noSERCA", "cru-edge-noSERCA"]
cru_center = [-200, -100, -150, 200, 150, 300]
cru_edge = [-1000, -1000, -1000, -600, 1000, 1000]
domains = [[], cru_center, cru_edge, [], cru_center, cru_edge]
for i in range(len(tests)):
    results_folder = results_folders[i]
    npy_dir = pathlib.Path(f"/root/shared/gitrepos/smart-comp-sci/ca2+-examples/npy-files")
    npy_dir.mkdir(exist_ok=True)
    tVec, results_all = smart_analysis.analyze_all(
        mesh_file=cru_mesh, results_path=results_folder, display=False,
        subdomain=domains[i])
    results_all.insert(0, tVec) # add time as first element in list
    max_length = len(tVec)
    for j in range(len(results_all)):
        if len(results_all[j]) > max_length:
            max_length = len(results_all[j])
    for j in range(len(results_all)):
        num_zeros = max_length - len(results_all[j])
        for k in range(num_zeros):
            results_all[j].append(0)
    np.save(npy_dir / f"{tests[i]}.npy", results_all)

In [ ]:
cur_dir = "/root/shared/gitrepos/smart-comp-sci/ca2+-examples"
cru_vars = ["ATP", "Ca", "CaSR", "CMDN", "CSQN", "RyR", "SERCA", "TRPN"]
plot_cru = "Ca"
f, (ax1, ax2) = plt.subplots(2, 1, sharex=False)
plot_cru1 = "Ca"
results_cru_withSERCA = np.load(f"{cur_dir}/npy-files-new/cru-center.npy")
results_cru_noSERCA = np.load(f"{cur_dir}/npy-files-new/cru-center-noSERCA.npy")
cru_idx1 = cru_vars.index(plot_cru1) + 1
ax1.plot(results_cru_withSERCA[0], results_cru_withSERCA[cru_idx1],label="CRU - with SERCA")
ax1.plot(results_cru_noSERCA[0], results_cru_noSERCA[cru_idx1],label="CRU - no SERCA")
# ax1.set_ylim([0, 2])
ax1.set_xlim([-.01, 0.2])
# ax1.legend()
ax1.set_ylabel("Calcium (μM)")
ax1.set_xlabel("Time (s)")

plot_cru2 = "CaSR"
cru_idx2 = cru_vars.index(plot_cru2) + 1
ax2.plot(results_cru_withSERCA[0], results_cru_withSERCA[cru_idx2],label="CRU - with SERCA")
ax2.plot(results_cru_noSERCA[0], results_cru_noSERCA[cru_idx2],label="CRU - no SERCA")
# ax2.legend()
ax2.set_ylabel("SR calcium (μM)")
ax2.set_ylim([0, 1400])
ax2.set_xlim([-.01, 0.2])
ax2.set_xlabel("Time (s)")
# plt.savefig("/root/shared/gitrepos/pyplots/cru_edge_plot.pdf", format="pdf")